In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from transformers import AutoImageProcessor, AutoModelForImageClassification
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2

In [3]:
object_det_model = YOLO("yolov8x.pt")

In [4]:
def rotate(img, angle):
    if angle == 0:
        return img
    h, w = img.size[1], img.size[0]
    m = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rad = np.radians(angle)
    nw = int(abs(np.sin(rad)) * h + abs(np.cos(rad)) * w)
    nh = int(abs(np.cos(rad)) * h + abs(np.sin(rad)) * w)
    m[0, 2] += (nw - w) / 2
    m[1, 2] += (nh - h) / 2
    return Image.fromarray(cv2.warpAffine(np.array(img), m, (nw, nh)))

In [5]:
def detect_best_class(model, img_path):
    img = Image.open(img_path).convert("RGB")
    angles = [0, 45, 90, 180]

    best = {"cls": None, "conf": 0}

    for a in angles:
        rimg = rotate(img, a)
        res = model(rimg)[0]

        if res.boxes:
            for b in res.boxes:
                conf = float(b.conf)
                cls = int(b.cls)

                if conf > best["conf"]:
                    best.update({"cls": cls, "conf": conf})

    if best["cls"] is None:
        return "None"

    return model.names[best["cls"]]

In [6]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.05):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        # Normalize 이후의 텐서에 노이즈를 더하기 때문에 std의 절대적인 크기에 주의
        noise = torch.randn(tensor.size()) * self.std + self.mean
        return torch.clamp(tensor + noise, -2., 2.)

In [7]:
# ImageNet 통계값 (표준적인 값)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    
    # 문서 이미지에 특화된 증강 추가: 원근 왜곡
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3, interpolation=transforms.InterpolationMode.BILINEAR),

    # 무작위 회전
    transforms.RandomChoice([
        transforms.RandomRotation(30),
        transforms.RandomRotation(45),
        transforms.RandomRotation(60),
        transforms.RandomRotation(75),
        transforms.RandomRotation(90),
        transforms.RandomRotation(120),
        transforms.RandomRotation(150),
        transforms.RandomRotation(180),
        transforms.RandomRotation(210),
        transforms.RandomRotation(240),
        transforms.RandomRotation(270),
        transforms.RandomRotation(300)
    ]),

    # 좌우 / 상하 반전
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    
    # 색상 지터링 추가
    transforms.ColorJitter(brightness=0.1, contrast=0.1),

    # 30% 확률로 흑백화
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.3),

    transforms.ToTensor(),
    
    # ImageNet 기반 정규화 추가 
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),

    # 가우시안 노이즈 추가
    AddGaussianNoise(0., 0.05),
])

# 검증 및 테스트용도 정규화 및 해상도 변경
test_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    # ImageNet 기반 정규화 추가 
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, label_encoder=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_encoder = label_encoder

        if 'target' in self.data.columns and self.label_encoder is not None:
            self.data['target'] = self.label_encoder.transform(self.data['target'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = f"{self.img_dir}/{self.data.iloc[idx, 0]}"
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if 'target' in self.data.columns:
            label = torch.tensor(self.data.iloc[idx, 1], dtype=torch.long)
            return image, label
        else:
            return image, -1

In [9]:
train_df = pd.read_csv("/home/realtheai/cv_competetion/data/train.csv")
le = LabelEncoder()
train_df['target'] = le.fit_transform(train_df['target'])

train_dataset = CustomImageDataset(
    csv_file="/home/realtheai/cv_competetion/data/train.csv",
    img_dir="/home/realtheai/cv_competetion/data/train",
    transform=train_transform,
    label_encoder=le
)

test_dataset = CustomImageDataset(
    csv_file="/home/realtheai/cv_competetion/data/sample_submission.csv",
    img_dir="/home/realtheai/cv_competetion/data/test",
    transform=test_transform,
    label_encoder=le
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)


In [10]:
processor = AutoImageProcessor.from_pretrained("facebook/convnext-base-384")
model = AutoModelForImageClassification.from_pretrained("facebook/convnext-base-384", num_labels=len(le.classes_), ignore_mismatched_sizes=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-base-384 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([17, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Epoch 증가: ConvNeXt 모델의 충분한 수렴을 위해 40 epoch으로 설정
num_epochs = 40

# tqdm으로 진행 상황 표시
from tqdm import tqdm
import sys

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    # 에포크 시작 메시지 (즉시 출력)
    print(f"\n🔄 Epoch {epoch+1}/{num_epochs} 시작...", flush=True)
    
    # tqdm으로 배치별 진행 표시
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100)
    
    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # 현재 배치 Loss 표시
        progress_bar.set_postfix({'batch_loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} 완료 - Avg Loss: {avg_loss:.4f}", flush=True)



🔄 Epoch 1/40 시작...


Epoch 1/40: 100%|██████████████████████████████| 197/197 [01:55<00:00,  1.71it/s, batch_loss=0.1911]

Epoch 1/40 완료 - Avg Loss: 1.5482

🔄 Epoch 2/40 시작...



Epoch 2/40: 100%|██████████████████████████████| 197/197 [01:54<00:00,  1.73it/s, batch_loss=0.5656]

Epoch 2/40 완료 - Avg Loss: 0.4928

🔄 Epoch 3/40 시작...


...

Epoch 38/40 완료 - Avg Loss: 0.0136

🔄 Epoch 39/40 시작...



Epoch 39/40: 100%|█████████████████████████████| 197/197 [01:31<00:00,  2.16it/s, batch_loss=0.0019]

Epoch 39/40 완료 - Avg Loss: 0.0220

🔄 Epoch 40/40 시작...



Epoch 40/40: 100%|█████████████████████████████| 197/197 [01:30<00:00,  2.17it/s, batch_loss=0.0002]

Epoch 40/40 완료 - Avg Loss: 0.0141


In [12]:
model.eval()
all_preds = []

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)

        for img_tensor in images:
            img_np = (img_tensor.cpu().permute(1, 2, 0).numpy() * 255).astype(np.uint8)
            pil_img = Image.fromarray(img_np)

            yolo_res = object_det_model(pil_img)[0]

            detected_car = False

            if yolo_res.boxes:
                for box in yolo_res.boxes:
                    cls = int(box.cls[0])
                    class_name = object_det_model.names[cls]

                    if class_name == "car":
                        detected_car = True
                        break

            if detected_car:
                all_preds.append(16)
            else:
                img_batch = img_tensor.unsqueeze(0).to(device)
                outputs = model(img_batch).logits
                pred_class = outputs.argmax(dim=1).cpu().item()
                all_preds.append(pred_class)

pred_labels = le.inverse_transform(all_preds)

result = pd.read_csv('/home/realtheai/cv_competetion/data/sample_submission.csv')
result['target'] = pred_labels
result.to_csv('v2_convnext_384_output.csv', index=False)
print("✅ 저장 완료: v2_ convnext_384_output.csv")


0: 640x640 1 cake, 1 dining table, 25.9ms
Speed: 5.4ms preprocess, 25.9ms inference, 9.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cakes, 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 16.0ms
Speed: 2.5ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 kite, 1 cake, 17.3ms
Speed: 3.0ms preprocess, 17.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 16.0ms
Speed: 2.4ms preprocess, 16.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 kites, 39.2ms
Speed: 3.1ms preprocess, 39.2ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 book, 19.9ms
Speed: 2.5ms preprocess, 19.9ms inference, 1.7ms pos